# Cancelamento de Clientes - Telco (dataset criado pela IBM para demonstração da ferramenta IBM Cognos Analytics)

### Contém informações sobre uma empresa fictícia de telecomunicações que forneceu serviços de telefonia residencial e internet para 7043 clientes na Califórnia no 3º trimestre.

### Etapa do pipeline - Realizado por Sabrina Otoni da Silva - 2024/01

### Objetivo: Testar combinações dos tratamentos desenvolvidos e ajustes nos modelos escolhidos para refinamento das predições. 

In [1]:
from pathlib import Path

import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import sys
import os

automations_dir = os.path.join(os.getcwd(), '../automations')

if automations_dir not in sys.path:
    sys.path.append(automations_dir)

from data_processing import LogTransformer, BoxCoxTransformer, RBFTransformer, KMeansCluster, DropColumns, ServiceTransformer, CategoricalEncoder

import warnings
warnings.filterwarnings('ignore')

Por questões de processamento, foram selecionados 3 modelos.

In [2]:
datapath = Path('../data')
csv_path = Path(f'{datapath}/d02_intermediate')
preprocesspath = Path('../preprocessing')

In [3]:
X_train = pd.read_csv(f'{csv_path}/X_train.csv')
y_train = pd.read_csv(f'{csv_path}/y_train.csv')

In [4]:
class PassthroughTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X

In [5]:
def get_kmeans():
    return KMeansCluster(model_path='../preprocessing/kmeans_model.pkl', columns_cluster=['Latitude', 'Longitude'])

In [6]:
class ConditionalServiceTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, encoder_type):
        self.service_transformer = ServiceTransformer(columns=['Multiple Lines', 'Online Security', 'Online Backup', 'Device Protection', 
                                                            'Tech Support', 'Streaming TV', 'Streaming Movies'])
        self.encoder_type = encoder_type

    def fit(self, X, y=None):
        if self.encoder_type == 'onehot':
            self.service_transformer.fit(X, y)
            return self
        else:
            return self

    def transform(self, X):
        if self.encoder_type == 'onehot':
            return self.service_transformer.transform(X)
        else:
            return X

In [7]:
def get_transformer(transformer_type: str, columns: list = None):
    if transformer_type == 'log':
        return LogTransformer(model_path='../preprocessing/log_transformer_model.pkl', columns=['Total Charges'])
    elif transformer_type == 'boxcox':
        return BoxCoxTransformer(model_path='../preprocessing/boxcox_transformer_model.pkl', columns=['Total Charges'])
    else:
        return PassthroughTransformer()

In [8]:
def get_rbf():
    return RBFTransformer(model_path='../preprocessing/rbf_transformer_model.pkl', column='Tenure Months') 

In [9]:
def build_pipeline(transformer, cond_encoder_type: str, cat_encoder_type: str, scaler, model):
    pipeline_steps = [
    ('kmeans_cluster', get_kmeans()),
    ('import_drop', DropColumns(drop_columns=["City", "Latitude", "Longitude", "ID"])),
    ('service_transformer', ConditionalServiceTransformer(encoder_type=cond_encoder_type)),
    ('categorical_encoder', CategoricalEncoder(encoder_type=cat_encoder_type, specified_columns=["Gender", "Senior Citizen", "Partner", "Dependents", "Phone Service", "Multiple Lines", "Internet Service",
                                                                  "Online Security", "Online Backup", "Device Protection", "Tech Support", "Streaming TV", "Streaming Movies",
                                                                  "Contract", "Paperless Billing", "Payment Method", "Cluster"])),
    ('transformation', transformer),        
    ('rbf', get_rbf()),                                                              
    ('scaler', scaler),
    ('model', model)
    ]
    return Pipeline(pipeline_steps)

In [10]:
scaler_dict = {
    'standard': StandardScaler(),
    'minmax': MinMaxScaler(),
    'robust': RobustScaler()
    }

In [11]:
model_dict = {
    'dummy': DummyClassifier(),
    'logistic_regression': LogisticRegression(),
    'svr': SVR(),
    'knn': KNeighborsClassifier(),
    'random_forest': RandomForestClassifier(),
    'xgboost': XGBClassifier()
    }

In [12]:
pipeline = build_pipeline(
    cond_encoder_type='label',
    cat_encoder_type='label',
    transformer=get_transformer('log'),
    scaler=scaler_dict['minmax'],
    model=model_dict['dummy']
)

In [13]:
param_grid = {
    'kmeans_cluster__active': [True],
    'service_transformer__encoder_type': ['onehot','label'], 
    'categorical_encoder__encoder_type': ['onehot', 'label'], 
    'transformation': [get_transformer('log'), get_transformer('boxcox')],
    'rbf__active': [True, False],
    'scaler': [scaler_dict['standard'], scaler_dict['minmax'], scaler_dict['robust']],
    'model': [model_dict['dummy'], model_dict['xgboost']]
    # 'model__strategy': ['stratified', 'most_frequent'],
    # 'model__C': [0.1, 1, 10],
}

In [14]:
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=StratifiedKFold().split(X_train, y_train), 
                           scoring='accuracy', verbose=2, error_score="raise")
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[CV] END categorical_encoder__encoder_type=onehot, kmeans_cluster__active=True, model=DummyClassifier(), rbf__active=True, scaler=StandardScaler(), service_transformer__encoder_type=onehot, transformation=LogTransformer(columns=['Total Charges'],
               model_path='../preprocessing/log_transformer_model.pkl'); total time=   0.3s
[CV] END categorical_encoder__encoder_type=onehot, kmeans_cluster__active=True, model=DummyClassifier(), rbf__active=True, scaler=StandardScaler(), service_transformer__encoder_type=onehot, transformation=LogTransformer(columns=['Total Charges'],
               model_path='../preprocessing/log_transformer_model.pkl'); total time=   0.2s
[CV] END categorical_encoder__encoder_type=onehot, kmeans_cluster__active=True, model=DummyClassifier(), rbf__active=True, scaler=StandardScaler(), service_transformer__encoder_type=onehot, transformation=LogTransformer(columns=['Total Charges'],
               model_path='../preprocessing/log_transformer_model.pkl'); to

GridSearchCV(cv=<generator object _BaseKFold.split at 0x00000256441D8820>,
             error_score='raise',
             estimator=Pipeline(steps=[('kmeans_cluster',
                                        KMeansCluster(columns_cluster=['Latitude',
                                                                       'Longitude'],
                                                      model_path='../preprocessing/kmeans_model.pkl')),
                                       ('import_drop',
                                        DropColumns(drop_columns=['City',
                                                                  'Latitude',
                                                                  'Longitude',
                                                                  'ID'])),
                                       ('service_transformer',
                                        Conditiona...
                         'scaler': [StandardScaler(), MinMaxScaler(),
                                    RobustScaler()],
                         'service_transformer__encoder_type': ['onehot',
                                                               'label'],
                         'transformation': [LogTransformer(columns=['Total '
                                                                    'Charges'],
                                                           model_path='../preprocessing/log_transformer_model.pkl'),
                                            BoxCoxTransformer(columns=['Total '
                                                                       'Charges'],
                                                              model_path='../preprocessing/boxcox_transformer_model.pkl')]},
             scoring='accuracy', verbose=2)

In [15]:
grid_search.best_score_

0.7870182555780934

In [16]:
grid_search.best_estimator_

Pipeline(steps=[('kmeans_cluster',
                 KMeansCluster(columns_cluster=['Latitude', 'Longitude'],
                               model_path='../preprocessing/kmeans_model.pkl')),
                ('import_drop',
                 DropColumns(drop_columns=['City', 'Latitude', 'Longitude',
                                           'ID'])),
                ('service_transformer',
                 ConditionalServiceTransformer(encoder_type='onehot')),
                ('categorical_encoder',
                 CategoricalEncoder(specified_colu...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [17]:
grid_search.best_params_

{'categorical_encoder__encoder_type': 'onehot',
 'kmeans_cluster__active': True,
 'model': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=None, n_jobs=None,
               num_parallel_tree=None, random_state=None, ...),
 'rbf__active': True,
 'scaler': StandardScaler(),
 'service_transformer__encoder_type': 'onehot',
 'transformation': LogTransformer(columns